Get all comments from TikTok SPD:

CONDUCT SENTIMENT ANALYSIS ON ALL COMMENTS OF SPD:

In [1]:
# This is our classification prompt
PROMPT =  "Conduct a sentiment analysis of the comments on TikTok Vidoes by the official social media profiles of German parliamentarian parties on TikTok. The comments are generated in German. Consider the tone, context, and language and emojis used in each comment to determine its sentiment.  Respond with '0' if the comment contains neutral sentiment, respond with '1' if the content contains positive sentiment, and '2' if the comment contains negative sentiments. No explanation is needed. Here are some examples to guide your analysis. The comment 'Das nenn ich mal eine gute Partei',  the sentiment should be categorized as positive, so 1. If the comment is 'Ich habe keine starken Gefühle zu diesem Thema', the sentiment should be catgeorized as neutral, so 0. An example for a negative sentiment comment is 'Wenn das böse ein Gesicht hat', so it should be categorizes as 2." 

In [2]:
import openai
from openai import OpenAI

In [3]:
#We define which model to use throughout
MODEL = 'gpt-3.5-turbo-0125' # We use the cheap model 
WAIT_TIME = 0.01 # Wait time between each request. This depends on the rate limit of the model used

client = OpenAI(
    api_key= "" #Set the API key. See the how-to guide for further instructions
)

In [4]:
# Simple function for analyzing a message with OpenAI
def analyze_message(text, temperature=0.2):
    try:
        response = client.chat.completions.create(
            model = MODEL, 
            temperature=temperature,
            messages=[
                    {"role": "system", "content": f"'{PROMPT}'"}, #The system instruction tells the bot how it is supposed to behave
                    {"role": "user", "content": f"'{text}'"} #This provides the text to be analyzed.
                ]
        )
        result = ''
        for choice in response.choices:
            result += choice.message.content
        print(f"{text}: {result}")
        return result 
    except Exception as e:
        # Not the prettiest solution; but easy
        print(f'Exception happened: {e}')
        return 'error'

In [5]:
analyze_message("'Euch würde ich nicht wählen!'")

'Euch würde ich nicht wählen!': 2


'2'

In [6]:
import pandas as pd

# Specify the path to the CSV file
file_path = '/Users/emma/Desktop/SPD_cleaned_data.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df)

      create_time                                               text  \
0      1706717485  Bei Scholz ist es immer so dass er alle paar M...   
1      1706711701  Bin kein Fan von Olaf, aber da hat er einfach ...   
2      1706716544  Schade das so reden von ihm so selten kommen. ...   
3      1706718385  das Olaf sich an die Rede von Merz überhaupt e...   
4      1706714834          Endlich zeigt der Herr Scholz Kampfgeist!   
...           ...                                                ...   
6528   1712336070  Politiker sind nicht unsere Herrscher. Sie sin...   
6529   1712515667                             intelligenter Beitrag.   
6530   1712212087  Ja mit Deiner Aussage stehst Du in bester Trad...   
6531   1712211681  Wer an irgendwelche kruden Corona-Verschwörung...   
6532   1714305617                        Immer SPD wählen 🗳️✌️✌️✌️✌️   

                 video_id create_time_readable  
0     7330263977479442432  2024-01-31 16:11:25  
1     7330263977479442432  2024-01-31

In [7]:
# We start by defining a new column with the result
df['sentiment'] = None

# Save the DataFrame to a CSV file
df.to_csv('/Users/emma/SEN_SPD.csv', compression='gzip', index=False)

In [8]:
import time 
i = 0
while(True):

    #Find all unprocessed lines; where 
    left = df.loc[df['sentiment'].isna()]
    
    #No lines left? Then we're done
    if len(left)==0:
        print("All done!")
        break
        
    #Take a random line
    line = left.sample()
    index = line.index.values[0]
    text = line['text'].values[0]
    
    print(f"There are {len(left)} left to process. Processing: {index}")
    
    #Wait for a bit, to not overload the API
    time.sleep(WAIT_TIME)
    
    #Analyze the specific line, chunk by chunk    
    result = analyze_message(text)
    df.loc[index,'sentiment'] = result 

    i+=1    
    #Save the result to persistent file
    df.to_csv('/Users/emma/Sentiment_SPD.csv.gz',compression='gzip',index=False)

There are 6533 left to process. Processing: 5072
Die wissen das so ist aber die AfD geben nicht zu AfD halt 😏 oder Trotzdem,Danke SPD: 2
There are 6532 left to process. Processing: 1709
Go derya go stargirl wow🫶🏻: 1
There are 6531 left to process. Processing: 6203
Bei dein Namen auch: 2
There are 6530 left to process. Processing: 4518
schon blöd dieses Internet wenn man solche Sachen einfach googeln kann 😂: 2
There are 6529 left to process. Processing: 2755
afd spaltet das Volk an die naiven afd Wähler denkt mal was diese Partei macht: 2
There are 6528 left to process. Processing: 4949
Komisch 😐 , dass er sich noch daran erinnern kann ?: 2
There are 6527 left to process. Processing: 1704
SPD ♥️♥️♥️♥️♥️🙏: 1
There are 6526 left to process. Processing: 6489
Keine Ahnung, du mit "Touristen" meinst. Menschen, die in diesem Land Urlaub machen wollen, tun dies für gewöhnlich nicht mit kriminellen Absichten.: 0
There are 6525 left to process. Processing: 219
💖💖💖👍: 1
There are 6524 left to proc

In [9]:
#Check if data is the way we want
print(df.head(10))

   create_time                                               text  \
0   1706717485  Bei Scholz ist es immer so dass er alle paar M...   
1   1706711701  Bin kein Fan von Olaf, aber da hat er einfach ...   
2   1706716544  Schade das so reden von ihm so selten kommen. ...   
3   1706718385  das Olaf sich an die Rede von Merz überhaupt e...   
4   1706714834          Endlich zeigt der Herr Scholz Kampfgeist!   
5   1706711136   Olaf einfach richtig gut heute. So geht Kanzler!   
6   1706710856                                Starke Rede Kanzler   
7   1706710587  Was für eine kraftvolle und gute Rede und Reak...   
8   1706719047  Kann jemand Herrn Scholz sagen, dass wir genau...   
9   1706715668  Vielen Dank und so wichtig. Warum kann Scholz ...   

              video_id create_time_readable sentiment  
0  7330263977479442432  2024-01-31 16:11:25         1  
1  7330263977479442432  2024-01-31 14:35:01         1  
2  7330263977479442432  2024-01-31 15:55:44         1  
3  7330263977479

In [10]:
# Export to CSV
df.to_csv('/Users/emma/SPD_done.csv',index=False)